# Quantum System Types in NetKet: Spins, Fermions, and Bosons

This notebook explores the different types of quantum systems supported by NetKet, including spin systems, fermionic systems, and bosonic systems. We'll learn how to set up, manipulate, and analyze each type.

## Table of Contents
1. [Introduction to Quantum Systems](#introduction)
2. [Spin Systems](#spin-systems)
3. [Fermionic Systems](#fermionic-systems)
4. [Bosonic Systems](#bosonic-systems)
5. [Mixed Systems](#mixed-systems)
6. [Custom Hilbert Spaces](#custom-hilbert)
7. [System Symmetries](#symmetries)
8. [Performance Considerations](#performance)
9. [Conclusion](#conclusion)

## 1. Introduction to Quantum Systems {#introduction}

NetKet supports various types of quantum many-body systems:

- **Spin Systems**: Discrete spin degrees of freedom (S=1/2, S=1, etc.)
- **Fermionic Systems**: Fermions with Pauli exclusion principle
- **Bosonic Systems**: Bosons with unlimited occupation
- **Mixed Systems**: Combinations of different particle types

Each system type has its own Hilbert space structure, operators, and physical properties.

In [ ]:
# Import necessary libraries
import netket as nk
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
from typing import Any
import warnings
warnings.filterwarnings('ignore')

# Configure JAX
jax.config.update('jax_platform_name', 'cpu')
key = jax.random.PRNGKey(42)

print(f"NetKet version: {nk.__version__}")
print(f"JAX version: {jax.__version__}")

## 2. Spin Systems {#spin-systems}

Spin systems are the most common quantum many-body systems in NetKet. They consist of spins with discrete values.

In [ ]:
# Spin-1/2 system (most common)
N = 8
spin_half = nk.hilbert.Spin(s=1/2, N=N)

print(f"Spin-1/2 system:")
print(f"  Number of sites: {spin_half.size}")
print(f"  Local dimension: {spin_half.local_size}")
print(f"  Total Hilbert space dimension: {spin_half.n_states}")
print(f"  Local states: {spin_half.local_states}")

# Generate random configurations
key, subkey = jax.random.split(key)
configs = spin_half.random_state(subkey, size=5)
print(f"\nSample configurations:")
for i, config in enumerate(configs):
    print(f"  Config {i+1}: {config}")

In [ ]:
# Higher spin systems
spin_systems = {
    'S=1/2': nk.hilbert.Spin(s=1/2, N=4),
    'S=1': nk.hilbert.Spin(s=1, N=4),
    'S=3/2': nk.hilbert.Spin(s=3/2, N=4),
    'S=2': nk.hilbert.Spin(s=2, N=4)
}

print("Comparison of different spin systems (N=4 sites):")
print("-" * 60)
for name, hilbert in spin_systems.items():
    print(f"{name:8} | Local dim: {hilbert.local_size:2} | Total dim: {hilbert.n_states:8} | States: {hilbert.local_states}")

# Demonstrate the exponential growth
sites = range(2, 10)
dimensions_half = [nk.hilbert.Spin(s=1/2, N=n).n_states for n in sites]
dimensions_one = [nk.hilbert.Spin(s=1, N=n).n_states for n in sites]

plt.figure(figsize=(10, 6))
plt.semilogy(sites, dimensions_half, 'o-', label='S=1/2')
plt.semilogy(sites, dimensions_one, 's-', label='S=1')
plt.xlabel('Number of sites')
plt.ylabel('Hilbert space dimension')
plt.title('Exponential Growth of Hilbert Space')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

In [ ]:
# Custom spin systems with constraints
# Example: Spin system with total magnetization constraint
N = 6
target_magnetization = 0  # Total Sz = 0

# Create unconstrained system first
unconstrained = nk.hilbert.Spin(s=1/2, N=N)

# Create constrained system
constrained = nk.hilbert.Spin(s=1/2, N=N, total_sz=target_magnetization)

print(f"Magnetization-constrained spin system (N={N}, Sz_total={target_magnetization}):")
print(f"  Unconstrained dimension: {unconstrained.n_states}")
print(f"  Constrained dimension: {constrained.n_states}")
print(f"  Reduction factor: {unconstrained.n_states / constrained.n_states:.1f}")

# Generate constrained configurations
key, subkey = jax.random.split(key)
constrained_configs = constrained.random_state(subkey, size=5)
print(f"\nSample constrained configurations (Sz_total = {target_magnetization}):")
for i, config in enumerate(constrained_configs):
    magnetization = jnp.sum(config)
    print(f"  Config {i+1}: {config} (Sz_total = {magnetization})")

## 3. Fermionic Systems {#fermionic-systems}

Fermionic systems obey the Pauli exclusion principle: each orbital can contain at most one fermion.

In [ ]:
# Basic fermionic system
N_orbitals = 6
fermions = nk.hilbert.Fock(n_max=1, N=N_orbitals)  # n_max=1 for fermions

print(f"Fermionic system:")
print(f"  Number of orbitals: {fermions.size}")
print(f"  Max occupation per orbital: 1 (Pauli exclusion)")
print(f"  Local dimension: {fermions.local_size}")
print(f"  Total Hilbert space dimension: {fermions.n_states}")
print(f"  Local states (occupation numbers): {fermions.local_states}")

# Generate random fermionic configurations
key, subkey = jax.random.split(key)
fermi_configs = fermions.random_state(subkey, size=5)
print(f"\nSample fermionic configurations:")
for i, config in enumerate(fermi_configs):
    n_particles = jnp.sum(config)
    print(f"  Config {i+1}: {config} (N_particles = {n_particles})")

In [ ]:
# Fermionic system with fixed particle number
N_orbitals = 8
N_particles = 4

fermions_fixed = nk.hilbert.Fock(n_max=1, N=N_orbitals, n_particles=N_particles)

print(f"Fermionic system with fixed particle number:")
print(f"  Orbitals: {N_orbitals}")
print(f"  Particles: {N_particles}")
print(f"  Unconstrained dimension: {nk.hilbert.Fock(n_max=1, N=N_orbitals).n_states}")
print(f"  Constrained dimension: {fermions_fixed.n_states}")

# This is equivalent to choosing N_particles orbitals out of N_orbitals
from math import comb
expected_dim = comb(N_orbitals, N_particles)
print(f"  Expected dimension (C({N_orbitals},{N_particles})): {expected_dim}")

# Generate configurations with fixed particle number
key, subkey = jax.random.split(key)
fixed_configs = fermions_fixed.random_state(subkey, size=5)
print(f"\nSample configurations with {N_particles} particles:")
for i, config in enumerate(fixed_configs):
    occupied_orbitals = jnp.where(config == 1)[0]
    print(f"  Config {i+1}: {config} (occupied: {list(occupied_orbitals)})")

In [ ]:
# Spinful fermions (electrons with spin up/down)
N_sites = 4  # Number of spatial sites
# Each site has two orbitals: spin-up and spin-down
N_orbitals = 2 * N_sites

# Create spinful fermionic system
spinful_fermions = nk.hilbert.Fock(n_max=1, N=N_orbitals)

print(f"Spinful fermionic system (Hubbard-like):")
print(f"  Spatial sites: {N_sites}")
print(f"  Total orbitals (sites × spins): {N_orbitals}")
print(f"  Hilbert space dimension: {spinful_fermions.n_states}")

# Helper function to interpret spinful configurations
def interpret_spinful_config(config, n_sites):
    """Interpret a spinful fermionic configuration"""
    config = np.array(config)
    up_orbitals = config[:n_sites]  # First half: spin-up
    down_orbitals = config[n_sites:]  # Second half: spin-down
    
    result = []
    for i in range(n_sites):
        if up_orbitals[i] == 1 and down_orbitals[i] == 1:
            result.append('↑↓')  # Doubly occupied
        elif up_orbitals[i] == 1:
            result.append('↑ ')  # Spin-up only
        elif down_orbitals[i] == 1:
            result.append(' ↓')  # Spin-down only
        else:
            result.append('  ')  # Empty
    return result

# Generate and interpret spinful configurations
key, subkey = jax.random.split(key)
spinful_configs = spinful_fermions.random_state(subkey, size=5)
print(f"\nSample spinful fermionic configurations:")
for i, config in enumerate(spinful_configs):
    interpretation = interpret_spinful_config(config, N_sites)
    n_up = jnp.sum(config[:N_sites])
    n_down = jnp.sum(config[N_sites:])
    print(f"  Config {i+1}: {config}")
    print(f"            Sites: {' '.join(interpretation)} (N↑={n_up}, N↓={n_down})")

## 4. Bosonic Systems {#bosonic-systems}

Bosonic systems allow multiple particles to occupy the same state, with no upper limit in principle (though we typically impose a cutoff).

In [ ]:
# Basic bosonic system
N_modes = 4
n_max = 3  # Maximum occupation per mode

bosons = nk.hilbert.Fock(n_max=n_max, N=N_modes)

print(f"Bosonic system:")
print(f"  Number of modes: {bosons.size}")
print(f"  Max occupation per mode: {n_max}")
print(f"  Local dimension: {bosons.local_size}")
print(f"  Total Hilbert space dimension: {bosons.n_states}")
print(f"  Local states (occupation numbers): {bosons.local_states}")

# Generate random bosonic configurations
key, subkey = jax.random.split(key)
boson_configs = bosons.random_state(subkey, size=5)
print(f"\nSample bosonic configurations:")
for i, config in enumerate(boson_configs):
    n_particles = jnp.sum(config)
    print(f"  Config {i+1}: {config} (N_particles = {n_particles})")

In [ ]:
# Bosonic system with fixed particle number
N_modes = 6
n_max = 4
N_particles = 8

bosons_fixed = nk.hilbert.Fock(n_max=n_max, N=N_modes, n_particles=N_particles)

print(f"Bosonic system with fixed particle number:")
print(f"  Modes: {N_modes}")
print(f"  Max occupation per mode: {n_max}")
print(f"  Total particles: {N_particles}")
print(f"  Unconstrained dimension: {bosons.n_states}")
print(f"  Constrained dimension: {bosons_fixed.n_states}")

# Generate configurations with fixed particle number
key, subkey = jax.random.split(key)
fixed_boson_configs = bosons_fixed.random_state(subkey, size=5)
print(f"\nSample configurations with {N_particles} bosons:")
for i, config in enumerate(fixed_boson_configs):
    total_particles = jnp.sum(config)
    print(f"  Config {i+1}: {config} (total = {total_particles})")

In [ ]:
# Compare fermionic vs bosonic systems
N_sites = 4
N_particles = 2

fermions_comp = nk.hilbert.Fock(n_max=1, N=N_sites, n_particles=N_particles)
bosons_comp = nk.hilbert.Fock(n_max=N_particles, N=N_sites, n_particles=N_particles)

print(f"Comparison: {N_particles} particles on {N_sites} sites")
print("-" * 50)
print(f"Fermions (Pauli exclusion):")
print(f"  Hilbert space dimension: {fermions_comp.n_states}")
print(f"Bosons (unlimited occupation):")
print(f"  Hilbert space dimension: {bosons_comp.n_states}")

# Show all possible fermionic configurations
print(f"\nAll fermionic configurations:")
fermi_basis = fermions_comp.all_states()
for i, config in enumerate(fermi_basis):
    occupied = [j for j, occ in enumerate(config) if occ == 1]
    print(f"  {i+1}: {config} (particles at sites {occupied})")

print(f"\nSample bosonic configurations:")
key, subkey = jax.random.split(key)
sample_bosons = bosons_comp.random_state(subkey, size=min(10, bosons_comp.n_states))
for i, config in enumerate(sample_bosons):
    print(f"  {i+1}: {config}")

## 5. Mixed Systems {#mixed-systems}

NetKet allows combining different types of quantum systems into composite Hilbert spaces.

In [ ]:
# Mixed system: spins + bosons
N_spins = 3
N_boson_modes = 2
boson_n_max = 2

# Create individual Hilbert spaces
spin_part = nk.hilbert.Spin(s=1/2, N=N_spins)
boson_part = nk.hilbert.Fock(n_max=boson_n_max, N=N_boson_modes)

# Combine them
mixed_system = spin_part * boson_part

print(f"Mixed spin-boson system:")
print(f"  Spin part: {N_spins} spins-1/2 (dim = {spin_part.n_states})")
print(f"  Boson part: {N_boson_modes} modes, n_max = {boson_n_max} (dim = {boson_part.n_states})")
print(f"  Combined system dimension: {mixed_system.n_states}")
print(f"  Expected (product): {spin_part.n_states * boson_part.n_states}")
print(f"  Total degrees of freedom: {mixed_system.size}")

# Generate mixed configurations
key, subkey = jax.random.split(key)
mixed_configs = mixed_system.random_state(subkey, size=3)
print(f"\nSample mixed configurations:")
for i, config in enumerate(mixed_configs):
    spin_config = config[:N_spins]
    boson_config = config[N_spins:]
    print(f"  Config {i+1}: {config}")
    print(f"            Spins: {spin_config}, Bosons: {boson_config}")

In [ ]:
# More complex mixed system: multiple fermion species + spins
N_sites = 3

# Two fermion species (e.g., different atoms)
fermion_A = nk.hilbert.Fock(n_max=1, N=N_sites, n_particles=2)  # 2 A-particles
fermion_B = nk.hilbert.Fock(n_max=1, N=N_sites, n_particles=1)  # 1 B-particle
spins = nk.hilbert.Spin(s=1/2, N=N_sites)  # Spins on each site

# Combine all subsystems
complex_mixed = fermion_A * fermion_B * spins

print(f"Complex mixed system:")
print(f"  Fermion A: {fermion_A.n_states} states")
print(f"  Fermion B: {fermion_B.n_states} states")
print(f"  Spins: {spins.n_states} states")
print(f"  Total dimension: {complex_mixed.n_states}")
print(f"  Expected: {fermion_A.n_states * fermion_B.n_states * spins.n_states}")

# Generate a sample configuration
key, subkey = jax.random.split(key)
complex_config = complex_mixed.random_state(subkey, size=1)[0]

# Parse the configuration
offset = 0
fermA_config = complex_config[offset:offset+N_sites]
offset += N_sites
fermB_config = complex_config[offset:offset+N_sites]
offset += N_sites
spin_config = complex_config[offset:offset+N_sites]

print(f"\nSample complex configuration:")
print(f"  Full config: {complex_config}")
print(f"  Fermion A:   {fermA_config} (particles at sites {jnp.where(fermA_config==1)[0]})")
print(f"  Fermion B:   {fermB_config} (particles at sites {jnp.where(fermB_config==1)[0]})")
print(f"  Spins:       {spin_config}")

## 6. Custom Hilbert Spaces {#custom-hilbert}

For specialized applications, you can create custom Hilbert spaces with specific constraints or structures.

In [ ]:
# Custom discrete Hilbert space
# Example: 3-level systems (qutrits)
N_qutrits = 4
qutrit_states = [-1, 0, 1]  # Three possible states per site

qutrits = nk.hilbert.DiscreteHilbert(local_states=qutrit_states, N=N_qutrits)

print(f"Custom qutrit system:")
print(f"  Number of qutrits: {qutrits.size}")
print(f"  Local states: {qutrits.local_states}")
print(f"  Local dimension: {qutrits.local_size}")
print(f"  Total dimension: {qutrits.n_states}")

# Generate qutrit configurations
key, subkey = jax.random.split(key)
qutrit_configs = qutrits.random_state(subkey, size=5)
print(f"\nSample qutrit configurations:")
for i, config in enumerate(qutrit_configs):
    print(f"  Config {i+1}: {config}")

In [ ]:
# Custom Hilbert space with complex constraints
# Example: Ising model with defects (some sites can be empty)

class IsingWithDefects(nk.hilbert.DiscreteHilbert):
    """Custom Hilbert space for Ising model with defects"""
    
    def __init__(self, N, defect_probability=0.1):
        # States: -1 (spin down), 0 (defect/empty), +1 (spin up)
        local_states = [-1, 0, 1]
        super().__init__(local_states=local_states, N=N)
        self.defect_probability = defect_probability
    
    def random_state(self, key, size=None):
        """Generate random states with controlled defect density"""
        if size is None:
            shape = (self.size,)
        else:
            shape = (size, self.size)
        
        key1, key2 = jax.random.split(key)
        
        # First decide which sites are defects
        defect_mask = jax.random.bernoulli(key1, self.defect_probability, shape)
        
        # For non-defect sites, choose spin up or down
        spins = jax.random.choice(key2, jnp.array([-1, 1]), shape)
        
        # Combine: defects are 0, others are ±1
        states = jnp.where(defect_mask, 0, spins)
        
        return states

# Create and test the custom Hilbert space
N_sites = 8
defect_prob = 0.2

ising_defects = IsingWithDefects(N_sites, defect_probability=defect_prob)

print(f"Ising model with defects:")
print(f"  Sites: {N_sites}")
print(f"  Defect probability: {defect_prob}")
print(f"  Local states: {ising_defects.local_states} (↓, defect, ↑)")
print(f"  Hilbert space dimension: {ising_defects.n_states}")

# Generate configurations
key, subkey = jax.random.split(key)
defect_configs = ising_defects.random_state(subkey, size=5)
print(f"\nSample configurations with defects:")
for i, config in enumerate(defect_configs):
    n_defects = jnp.sum(config == 0)
    n_up = jnp.sum(config == 1)
    n_down = jnp.sum(config == -1)
    print(f"  Config {i+1}: {config} (↑:{n_up}, ↓:{n_down}, defects:{n_defects})")

## 7. System Symmetries {#symmetries}

Many quantum systems have symmetries that can be exploited to reduce computational complexity.

In [ ]:
# Translation symmetry example
N = 6
hilbert = nk.hilbert.Spin(s=1/2, N=N)
lattice = nk.graph.Chain(length=N, pbc=True)

# Define translation group
translations = []
for i in range(N):
    # Translation by i sites
    perm = [(j, (j + i) % N) for j in range(N)]
    translations.append(perm)

# Create symmetry group
translation_group = nk.group.PermutationGroup(translations, acting_on=range(N))

print(f"Translation symmetry group:")
print(f"  Number of translations: {len(translation_group)}")
print(f"  Group elements (first few):")
for i, elem in enumerate(translation_group.elems[:3]):
    print(f"    Translation {i}: {elem}")

# Test symmetry on a configuration
test_config = jnp.array([1, -1, 1, -1, 1, -1])
print(f"\nTesting translation symmetry:")
print(f"Original config: {test_config}")

for i in range(3):
    translated = translation_group.elems[i] @ test_config
    print(f"Translation by {i}: {translated}")

In [ ]:
# Point group symmetries (reflection, inversion)
N = 4
hilbert = nk.hilbert.Spin(s=1/2, N=N)

# Define symmetry operations
symmetries = []

# Identity
identity = [(i, i) for i in range(N)]
symmetries.append(identity)

# Reflection (for 1D chain: 0↔3, 1↔2)
reflection = [(0, 3), (1, 2), (2, 1), (3, 0)]
symmetries.append(reflection)

# Create point group
point_group = nk.group.PermutationGroup(symmetries, acting_on=range(N))

print(f"Point group symmetries:")
print(f"  Number of operations: {len(point_group)}")
print(f"  Operations: Identity, Reflection")

# Test on a configuration
test_config = jnp.array([1, -1, -1, 1])
print(f"\nTesting point group symmetry:")
print(f"Original:   {test_config}")
reflected = point_group.elems[1] @ test_config
print(f"Reflected:  {reflected}")
print(f"Symmetric: {jnp.array_equal(test_config, reflected)}")

# Try a symmetric configuration
symmetric_config = jnp.array([1, -1, -1, 1])
reflected_sym = point_group.elems[1] @ symmetric_config
print(f"\nSymmetric config: {symmetric_config}")
print(f"After reflection: {reflected_sym}")
print(f"Is symmetric: {jnp.array_equal(symmetric_config, reflected_sym)}")

## 8. Performance Considerations {#performance}

Different Hilbert space types have different computational characteristics and memory requirements.

In [ ]:
# Performance comparison: different system types
import time

N = 10
n_samples = 1000

systems = {
    'Spin-1/2': nk.hilbert.Spin(s=1/2, N=N),
    'Spin-1': nk.hilbert.Spin(s=1, N=N),
    'Fermions': nk.hilbert.Fock(n_max=1, N=N),
    'Bosons (n_max=2)': nk.hilbert.Fock(n_max=2, N=N),
    'Mixed (spin+boson)': nk.hilbert.Spin(s=1/2, N=N//2) * nk.hilbert.Fock(n_max=1, N=N//2)
}

print(f"Performance comparison (N={N}, {n_samples} samples):")
print("-" * 70)
print(f"{'System':<20} | {'Dimension':<12} | {'Sample Time':<12} | {'Memory (MB)':<12}")
print("-" * 70)

for name, system in systems.items():
    # Time sampling
    key, subkey = jax.random.split(key)
    start_time = time.time()
    samples = system.random_state(subkey, size=n_samples)
    sample_time = time.time() - start_time
    
    # Estimate memory usage (rough)
    memory_mb = samples.nbytes / 1024 / 1024
    
    print(f"{name:<20} | {system.n_states:<12} | {sample_time:<12.4f} | {memory_mb:<12.2f}")

print("-" * 70)

In [ ]:
# Scaling analysis: how dimensions grow with system size
sizes = range(2, 12)
scaling_data = {}

for N in sizes:
    scaling_data[N] = {
        'Spin-1/2': nk.hilbert.Spin(s=1/2, N=N).n_states,
        'Spin-1': nk.hilbert.Spin(s=1, N=N).n_states,
        'Fermions': nk.hilbert.Fock(n_max=1, N=N).n_states,
        'Bosons (n_max=2)': nk.hilbert.Fock(n_max=2, N=N).n_states if N <= 8 else np.nan
    }

# Plot scaling
plt.figure(figsize=(12, 8))

for system_type in ['Spin-1/2', 'Spin-1', 'Fermions', 'Bosons (n_max=2)']:
    dims = [scaling_data[N][system_type] for N in sizes]
    valid_sizes = [N for N, dim in zip(sizes, dims) if not np.isnan(dim)]
    valid_dims = [dim for dim in dims if not np.isnan(dim)]
    
    plt.semilogy(valid_sizes, valid_dims, 'o-', label=system_type, linewidth=2, markersize=6)

plt.xlabel('System Size (N)', fontsize=12)
plt.ylabel('Hilbert Space Dimension', fontsize=12)
plt.title('Hilbert Space Scaling with System Size', fontsize=14)
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.xticks(sizes)

# Add exponential reference lines
x_ref = np.array(sizes)
plt.semilogy(x_ref, 2**x_ref, '--', alpha=0.5, color='gray', label='2ᴺ (spin-1/2)')
plt.semilogy(x_ref, 3**x_ref, '--', alpha=0.5, color='red', label='3ᴺ (spin-1)')

plt.legend(fontsize=10)
plt.tight_layout()
plt.show()

print("Key observations:")
print("- Spin systems scale exponentially: 2ᴺ for spin-1/2, 3ᴺ for spin-1")
print("- Fermions have same dimension as spin-1/2 (both are binary occupation)")
print("- Bosons scale even faster due to multiple occupation possibilities")
print("- Constraints (fixed particle number) can dramatically reduce dimensions")

## 9. Conclusion {#conclusion}

This notebook has provided a comprehensive overview of the different quantum system types supported by NetKet:

### Key System Types:

1. **Spin Systems**:
   - Most common in quantum magnetism
   - Support for any spin value (1/2, 1, 3/2, etc.)
   - Can include constraints (total magnetization)
   - Exponential scaling: $(2S+1)^N$

2. **Fermionic Systems**:
   - Pauli exclusion principle
   - Useful for electronic systems
   - Support for fixed particle number
   - Can model spinful fermions

3. **Bosonic Systems**:
   - Unlimited occupation (with practical cutoff)
   - Relevant for cold atoms, photons
   - Fastest dimensional growth
   - Support for particle number conservation

4. **Mixed Systems**:
   - Combine different particle types
   - Model complex quantum materials
   - Dimensions multiply

### Practical Considerations:

- **Scaling**: Hilbert space dimensions grow exponentially
- **Constraints**: Can dramatically reduce computational cost
- **Symmetries**: Exploit to improve efficiency
- **Memory**: Larger local dimensions require more memory
- **Performance**: Simple systems (spin-1/2) are most efficient

### Best Practices:

1. **Choose appropriate system type** for your physics
2. **Use constraints** when physically meaningful
3. **Exploit symmetries** to reduce computational cost
4. **Monitor memory usage** for large systems
5. **Start simple** before moving to complex mixed systems

### Applications:

- **Quantum magnetism**: Spin systems
- **Electronic materials**: Fermionic systems
- **Cold atom physics**: Bosonic systems
- **Quantum materials**: Mixed systems
- **Quantum computing**: Custom discrete systems

Understanding these different system types and their properties is crucial for effectively using NetKet to study quantum many-body physics. The choice of system type significantly impacts both the physics you can explore and the computational resources required.